# 🚀 MNIST Model Training với Data Augmentation - Google Colab

## 💡 Mục đích
Train MNIST model với **Data Augmentation** để giải quyết vấn đề **Domain Gap**.

### ⚠️ Vấn đề Domain Gap:
- **Model cũ**: Chỉ học trên MNIST (ảnh sạch, căn giữa hoàn hảo)
- **Ảnh thực tế**: Bị lệch, xoay, zoom, nét mỏng, ánh sáng không đều
- **Kết quả**: Model dự đoán sai!

### ✅ Giải pháp:
**Data Augmentation** - Làm "bẩn" ảnh MNIST khi train:
- Xoay ngẫu nhiên ±15°
- Dịch chuyển ngẫu nhiên 15%
- Zoom in/out ngẫu nhiên 15%
- Làm méo ảnh (shear)

→ Model sẽ "khoan dung" hơn với ảnh viết tay thực tế!


## 📦 Bước 1: Kiểm tra GPU và Import thư viện


In [ ]:
# Kiểm tra GPU
import tensorflow as tf
print(f"TensorFlow version: {tf.__version__}")
print(f"GPU available: {tf.config.list_physical_devices('GPU')}")

if tf.config.list_physical_devices('GPU'):
    print("✅ GPU được kích hoạt - Training sẽ nhanh!")
else:
    print("⚠️  Chỉ có CPU - Hãy bật GPU: Runtime > Change runtime type > GPU")


In [ ]:
# Import các thư viện cần thiết
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

print("✅ Import thành công!")


## 📥 Bước 2: Load và Preprocess MNIST Dataset


In [ ]:
# Load MNIST dataset
print("📥 Đang tải MNIST dataset...")
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

print(f"Train shape: {x_train.shape}")
print(f"Test shape: {x_test.shape}")

# Hiển thị một vài mẫu
fig, axes = plt.subplots(1, 10, figsize=(15, 2))
for i, ax in enumerate(axes):
    ax.imshow(x_train[i], cmap='gray')
    ax.set_title(f"Label: {y_train[i]}")
    ax.axis('off')
plt.suptitle('MNIST Original Samples', fontsize=16)
plt.tight_layout()
plt.show()


In [ ]:
# Preprocess data
print("🔄 Preprocessing data...")

# Reshape và normalize về [0, 1]
x_train = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
x_test = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0

# Convert labels to categorical (one-hot encoding)
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

print(f"✅ X_train: {x_train.shape}, Y_train: {y_train.shape}")
print(f"✅ X_test: {x_test.shape}, Y_test: {y_test.shape}")


## 🎨 Bước 3: Tạo Data Augmentation Generator

**ĐÂY LÀ PHẦN QUAN TRỌNG NHẤT!** 🔥

Data Augmentation sẽ làm cho ảnh MNIST "giống" ảnh viết tay thực tế hơn.


In [ ]:
# Tạo Data Augmentation Generator
print("🎨 Tạo Data Augmentation Generator...")

datagen = ImageDataGenerator(
    rotation_range=15,       # Ngẫu nhiên xoay +/- 15 độ
    width_shift_range=0.15,  # Ngẫu nhiên dịch ngang 15%
    height_shift_range=0.15, # Ngẫu nhiên dịch dọc 15%
    zoom_range=0.15,         # Ngẫu nhiên phóng to/thu nhỏ 15%
    shear_range=0.1,         # Ngẫu nhiên làm méo ảnh (nghiêng)
    fill_mode='constant',    # Fill phần trống bằng màu đen (0)
    cval=0
)

# Fit datagen vào training data
datagen.fit(x_train)

print("✅ Data Augmentation Generator ready!")
print("   → Model sẽ học từ ảnh đã được 'làm bẩn' (augmented)")
print("   → Model sẽ khoan dung hơn với ảnh thực tế!")


In [ ]:
# Demo: Hiển thị ảnh gốc vs ảnh đã augmented
print("📊 Demo: So sánh ảnh gốc vs ảnh augmented")

# Lấy 1 ảnh mẫu
sample_img = x_train[0:1]  # Shape (1, 28, 28, 1)
sample_label = np.argmax(y_train[0])

# Tạo augmented images
fig, axes = plt.subplots(2, 5, figsize=(12, 5))

# Hàng 1: Ảnh gốc
for i in range(5):
    axes[0, i].imshow(sample_img[0].reshape(28, 28), cmap='gray')
    axes[0, i].set_title('Original')
    axes[0, i].axis('off')

# Hàng 2: Ảnh augmented
aug_iter = datagen.flow(sample_img, batch_size=1)
for i in range(5):
    aug_img = next(aug_iter)[0]
    axes[1, i].imshow(aug_img.reshape(28, 28), cmap='gray')
    axes[1, i].set_title('Augmented')
    axes[1, i].axis('off')

plt.suptitle(f'Data Augmentation Demo (Label: {sample_label})', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("👀 Quan sát: Ảnh augmented bị xoay, dịch, zoom, méo → Giống ảnh thực tế hơn!")


## 🏗️ Bước 4: Build CNN Model


In [ ]:
# Tạo CNN Model
print("🏗️  Building CNN Model...")

model = keras.Sequential([
    # Input layer
    layers.Input(shape=(28, 28, 1)),
    
    # Convolutional Block 1
    layers.Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Dropout(0.25),
    
    # Convolutional Block 2
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Dropout(0.25),
    
    # Convolutional Block 3
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'),
    layers.Dropout(0.25),
    
    # Fully Connected Layers
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')
])

# Compile model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Hiển thị summary
model.summary()

print("\n✅ Model đã sẵn sàng!")


## 🎯 Bước 5: Training với Data Augmentation

**LƯU Ý:**
- Tăng `epochs` lên **30** (thay vì 15-20) vì model phải học bài toán khó hơn
- Sử dụng `datagen.flow()` thay vì truyền trực tiếp `x_train`
- Training time: ~15-20 phút trên GPU Colab


In [ ]:
# Thiết lập callbacks
callbacks = [
    keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True,
        verbose=1
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        verbose=1
    )
]

print("✅ Callbacks configured!")


In [ ]:
# 🚀 TRAINING với Data Augmentation!
print("="*70)
print("🎯 BẮT ĐẦU TRAINING VỚI DATA AUGMENTATION")
print("="*70)
print("⏰ Ước tính: 15-20 phút trên GPU Colab")
print("")

EPOCHS = 30
BATCH_SIZE = 128

history = model.fit(
    # ⚠️ QUAN TRỌNG: Sử dụng datagen.flow() thay vì x_train trực tiếp
    datagen.flow(x_train, y_train, batch_size=BATCH_SIZE),
    
    epochs=EPOCHS,
    
    # Validation data KHÔNG augment (giữ nguyên)
    validation_data=(x_test, y_test),
    
    # Số steps mỗi epoch
    steps_per_epoch=len(x_train) // BATCH_SIZE,
    
    callbacks=callbacks,
    verbose=1
)

print("\n" + "="*70)
print("✅ TRAINING HOÀN THÀNH!")
print("="*70)


## 📊 Bước 6: Đánh giá và Visualize kết quả


In [ ]:
# Evaluate trên test set
print("📊 Đánh giá model trên test set...")
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)

print("\n" + "="*70)
print("📈 KẾT QUẢ CUỐI CÙNG")
print("="*70)
print(f"✅ Test Accuracy: {test_acc:.4f} ({test_acc*100:.2f}%)")
print(f"✅ Test Loss: {test_loss:.4f}")
print("="*70)


In [ ]:
# Plot training history
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy plot
ax1.plot(history.history['accuracy'], label='Train Accuracy', linewidth=2)
ax1.plot(history.history['val_accuracy'], label='Val Accuracy', linewidth=2)
ax1.set_title('Model Accuracy', fontsize=14, fontweight='bold')
ax1.set_xlabel('Epoch', fontsize=12)
ax1.set_ylabel('Accuracy', fontsize=12)
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3)

# Loss plot
ax2.plot(history.history['loss'], label='Train Loss', linewidth=2)
ax2.plot(history.history['val_loss'], label='Val Loss', linewidth=2)
ax2.set_title('Model Loss', fontsize=14, fontweight='bold')
ax2.set_xlabel('Epoch', fontsize=12)
ax2.set_ylabel('Loss', fontsize=12)
ax2.legend(fontsize=11)
ax2.grid(True, alpha=0.3)

plt.suptitle('Training History với Data Augmentation', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print("✅ Training history plotted!")


In [ ]:
# Test predictions với một vài mẫu ngẫu nhiên
print("🧪 Testing predictions trên một vài mẫu ngẫu nhiên...\n")

# Chọn 10 ảnh ngẫu nhiên
indices = np.random.choice(len(x_test), 10, replace=False)
sample_images = x_test[indices]
sample_labels = y_test[indices]

# Predict
predictions = model.predict(sample_images, verbose=0)

# Visualize
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
axes = axes.ravel()

for i in range(10):
    true_label = np.argmax(sample_labels[i])
    pred_label = np.argmax(predictions[i])
    confidence = predictions[i][pred_label] * 100
    
    # Hiển thị ảnh
    axes[i].imshow(sample_images[i].reshape(28, 28), cmap='gray')
    
    # Màu xanh nếu đúng, đỏ nếu sai
    color = 'green' if true_label == pred_label else 'red'
    axes[i].set_title(f"True: {true_label}\nPred: {pred_label}\n({confidence:.1f}%)", 
                      color=color, fontsize=10, fontweight='bold')
    axes[i].axis('off')

plt.suptitle('Predictions Test (Green=Correct, Red=Wrong)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# In ra accuracy
correct = sum([np.argmax(sample_labels[i]) == np.argmax(predictions[i]) for i in range(10)])
print(f"\n✅ Accuracy trên 10 mẫu: {correct}/10 ({correct*10}%)")


## 💾 Bước 7: Lưu Model


In [ ]:
# Lưu model
model_path = 'mnist_model_augmented.h5'
model.save(model_path)

print(f"✅ Model đã được lưu tại: {model_path}")
print(f"📦 Kích thước file: {os.path.getsize(model_path) / (1024*1024):.2f} MB")


## 📥 Bước 8: Download Model về máy Local

**Chạy cell bên dưới để download model về máy:**


## 📥 Bước 8: Download Model về máy Local

**Chạy cell bên dưới để download model về máy:**


In [ ]:
# Download model về máy local
from google.colab import files

print("📥 Đang chuẩn bị download model...")
files.download(model_path)
print("✅ Download hoàn tất!")
print("\n💡 Hướng dẫn:")
print("   1. File sẽ được tải về thư mục Downloads")
print("   2. Di chuyển file 'mnist_model_augmented.h5' vào thư mục 'models/' của project")
print("   3. Đổi tên thành 'mnist_model.h5' (hoặc update code app.py)")
print("   4. Chạy lại Streamlit app: streamlit run app.py")
print("   5. Enjoy! 🎉")


## 🎉 HOÀN THÀNH!

### 📋 Tóm tắt:
- ✅ Đã train model với **Data Augmentation**
- ✅ Model đã học từ ảnh bị xoay, dịch, zoom, méo
- ✅ Model sẽ **khoan dung hơn** với ảnh viết tay thực tế
- ✅ Đã lưu và download model về máy

### 🚀 Tiếp theo:
1. Di chuyển model vào thư mục `models/` của project
2. Chạy Streamlit app: `streamlit run app.py`
3. Test với ảnh viết tay của bạn!

### 💡 Kỳ vọng:
- Accuracy trên MNIST test set: **~99%**
- Accuracy trên ảnh viết tay thực tế: **Tăng đáng kể** so với model cũ!

---

**Chúc mừng bạn đã giải quyết vấn đề Domain Gap! 🎊**
